In [2]:
import config
import numpy as np
import scipy.io as scio
import torch
import matplotlib.pyplot as plt
from model import Transformer
from scipy import interpolate

/mnt/DATA1/wuliubin/miniconda3/envs/DRILT/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Type: GSP, QGC, VD
# For the data of VD, its raw data must be downloaded first from https://www.escholar.manchester.ac.uk/uk-ac-man-scw:303820'.

Type = "QGC"
NmrData = scio.loadmat("./Dataset/" + Type + "_net_input.mat")

scale = 1.0  # Parameters used to adjust the calculation range of D. When the required value of the diffusion coefficient exceeds 
# the computational boundary, increasing this parameter enables obtaining a complete output result. However, it is important to 
# note that this may affect the processing effect of data with diffusion coefficient values that are close to each other.

if Type == 'VD':    
    with torch.no_grad():
        module = Transformer.load_from_checkpoint("Result/VD/last.ckpt")
        module.cpu()
        module.eval()
        module.freeze()
    test_input = NmrData['S'][np.newaxis, :, :]    

else:
    with torch.no_grad():
        module = Transformer.load_from_checkpoint("Result/DOSY/last.ckpt")
        module.cpu()
        module.eval()
        module.freeze()

    NmrDatai = np.zeros([NmrData['S'].shape[0], module.linear_en.in_features])
    for i in np.arange(NmrData['S'].shape[0]):
        f = interpolate.interp1d(NmrData['b'][0]*scale, NmrData['S'][i, :], fill_value='extrapolate')
        NmrDatai[i] = f(np.linspace(0, np.max(NmrData['b'][0]), module.linear_en.in_features))
    test_input = NmrDatai[np.newaxis, :, :]
    
test_input = torch.tensor(test_input)
test_input = test_input.to(torch.float32)

test_out = module(test_input)/config.mul_label

test_out = test_out * test_input[:, :, 0][:, :, np.newaxis]
test_out = test_out[0].cpu().detach().numpy()
test_out[test_out < np.tile(((np.max(test_out, axis=1)) * 0.7)[:, np.newaxis], [1, module.hparams.label_size])] = 0

scio.savemat("./Result/" + Type + "_Result.mat", {"Z":test_out, "calculated_max_D":14*(config.max_b*scale/NmrData['b'][0][-1])})

In [3]:
# Type: T1, T2

Type = "T1"
NmrData = scio.loadmat("./Dataset/" + Type + "_net_input.mat")

with torch.no_grad():
    module = Transformer.load_from_checkpoint("Result/T1T2/last.ckpt")
    module.cpu()
    module.eval()
    module.freeze()

test_out = np.zeros([NmrData['S'].shape[0], module.hparams.label_size])
for i in np.arange(NmrData['S'].shape[0]):
    test_input = np.reshape(NmrData['S'][i, :], (1, module.hparams.n_fre, module.hparams.signal_dim))
    test_input = torch.tensor(test_input).to(torch.float32)

    out = module(test_input)/config.mul_label
    out = out[0, 0].cpu().detach().numpy()
    test_out[i, :] = out

scio.savemat("./Result/" + Type + "_Result.mat", {"Z":test_out})